# Optimizing model parameters

## 1. Data load and build model

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [5]:
training_data = datasets.FashionMNIST(
    root="./",
    train=True,
    download=True,
    transform=ToTensor()
)

In [6]:
test_data = datasets.FashionMNIST(
    root="./",
    train=False,
    download=True,
    transform=ToTensor()
)

In [7]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = NeuralNetwork()

In [10]:
# Hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [14]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

In [15]:
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 2. Optimization

In [19]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [21]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.162158  [    0/60000]
loss: 2.149028  [ 6400/60000]
loss: 2.102813  [12800/60000]
loss: 2.125769  [19200/60000]
loss: 2.071686  [25600/60000]
loss: 2.009329  [32000/60000]
loss: 2.036908  [38400/60000]
loss: 1.956663  [44800/60000]
loss: 1.960252  [51200/60000]
loss: 1.901116  [57600/60000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.895344 

Epoch 2
-------------------------------
loss: 1.927802  [    0/60000]
loss: 1.893614  [ 6400/60000]
loss: 1.788570  [12800/60000]
loss: 1.831208  [19200/60000]
loss: 1.732281  [25600/60000]
loss: 1.670604  [32000/60000]
loss: 1.688879  [38400/60000]
loss: 1.587243  [44800/60000]
loss: 1.612317  [51200/60000]
loss: 1.509562  [57600/60000]
Test Error: 
 Accuracy: 62.5%, Avg loss: 1.528053 

Epoch 3
-------------------------------
loss: 1.595912  [    0/60000]
loss: 1.552739  [ 6400/60000]
loss: 1.411792  [12800/60000]
loss: 1.485702  [19200/60000]
loss: 1.375539  [25600/60000]
loss: 1.355708  [32000/600